# LC3 DATA ANALYSIS

In this notebook we analize the relation of the compressive strength of LC3 with the clay used for its elaboration properties. For this purpose we utilize the dataset given by the Constructions Materials Laboratory at EPFL. This dataset consists of different measures of compressive strength for different types of LC3 cement in which the clay used for their ellaboration varies. We have also several clay properties measured and the objetive is finding the relation of the compressive strength (CS) with these for a deeper understanding of key elements in LC3 cement.

### Load libraries and data

In [ ]:
from lc3_implementations import *

In [ ]:
# Input file path (can be also an excel)
DATA_MERGED_PATH = './data/data_merged.ods'
DATA_FULL_PATH   = './data/data_full.ods'

In [ ]:
# Read full data and remove empty lines
data_full = pd.read_excel(DATA_FULL_PATH,sheet_name='Clays_CS')
data_full.dropna(how="all", inplace=True)

In [ ]:
# Read clay properties
data_clay   = pd.read_excel(DATA_FULL_PATH,sheet_name='Clays_properties')

In [ ]:
# Merge to have the whole dataset
data_full_clay = pd.merge(data_full, data_clay, left_on='Clay', right_on='Clay', how='left')

In [ ]:
# We rename some columns for having an easier reference
data_full_clay.rename(columns = {'Calcined kaolinite content (%)':'Kaolinite_content'}, inplace = True)
data_full_clay.rename(columns = {'Dv,50 (µm)':'Dv50'                                 }, inplace = True)
data_full_clay.rename(columns = {'BET Specific surface (m2/g)':'BET_specific_surface'}, inplace = True)

data_full_clay.rename(columns = {'STD'  : 'STD_1D'}, inplace = True)
data_full_clay.rename(columns = {'STD.1': 'STD_3D'}, inplace = True)
data_full_clay.rename(columns = {'STD.2': 'STD_7D'}, inplace = True)
data_full_clay.rename(columns = {'STD.3':'STD_28D'}, inplace = True)
data_full_clay.rename(columns = {'STD.4':'STD_90D'}, inplace = True)

# Sorting allows us to plot functions more easily
data_full_clay = data_full_clay.sort_values('Kaolinite_content')

# Get useful dataframe information
#data               # Get data
#data.describe()    # Get data general information
#data.columns       # Get data features names
#data.corr()        # Get correlation matrix

### Visualize data

In [ ]:
# Take kaolinite content
x = data_full_clay['Kaolinite_content'].values

# Take compression strengths
y1  = data_full_clay[ "day_1"].values
y3  = data_full_clay[ "day_3"].values
y7  = data_full_clay[ "day_7"].values
y28 = data_full_clay["day_28"].values
y90 = data_full_clay["day_90"].values

# Measures have missing values
x1 =   x[np.logical_not(np.isnan(y1))]
y1 = y1[np.logical_not(np.isnan(y1))]
x3 =   x[np.logical_not(np.isnan(y3))]
y3 = y3[np.logical_not(np.isnan(y3))]
x7 =   x[np.logical_not(np.isnan(y7))]
y7 = y7[np.logical_not(np.isnan(y7))]
x28 =   x[np.logical_not(np.isnan(y28))]
y28 = y28[np.logical_not(np.isnan(y28))]
x90 =   x[np.logical_not(np.isnan(y90))]
y90 = y90[np.logical_not(np.isnan(y90))]

# Take standard deviations
#z1  = data[ "STD_1D"].values
#z3  = data[ "STD_3D"].values
#z7  = data[ "STD_7D"].values
#z28 = data["STD_28D"].values
#z90 = data["STD_90D"].values

In [ ]:
# Show points using matplotlib.pyplot library
fig, ax = plt.subplots(figsize=(12, 8))
plt.plot(x1,y1,'c^',x3,y3,'bs',x7,y7,'r^',x28,y28,'go', x90,y90,'m^' )
plt.xlabel('%Kaolinite Content')
plt.ylabel('Compressive Strenght')

d1_patch  = mpatches.Patch(color='cyan',      label='After  1 day')
d3_patch  = mpatches.Patch(color='blue',      label='After  3 days')
d7_patch  = mpatches.Patch(color='red',       label='After  7 days')
d28_patch = mpatches.Patch(color='darkgreen', label='After 28 days')
d90_patch = mpatches.Patch(color='purple',    label='After 90 days')
plt.legend(handles=[d1_patch,d3_patch,d7_patch,d28_patch,d90_patch])

plt.show()

# COMPRESSION STRENGTH (CS)

### Linear regression models

We have seen that the kaolinite content is the most predictive feature that we have. We are going to start creating simple linear regression models and then, following the appreciations obtained by visualizing the data, we are going to create non-linear models based on the kaolinite content for better fitting the data distribution as well as models with more features for avoiding data sparsification.

Two metrics are going to be extremely important here:
* **R square:** Is giving us a measurement of how good is our model (the closer to 1.0 the better). 
* **Validation score:** Is going to let us control overfitting. Improving R² means nothing if validation is worse. We'll use mean squared error with Leave One Out cross validation to estimate it.

In [ ]:
# TODO: Remove outlier in y1 data?
# TODO: Idea for the report, removing pesimist outliers is not a good idea in our project!
# TODO: Add OPC compression strength values for reference

#### Linear models based on the kaolinite content

In [ ]:
leave_one_out_validation(x1.reshape(-1,1), y1)

In [ ]:
leave_one_out_validation(x3.reshape(-1,1), y3)

In [ ]:
leave_one_out_validation(x7.reshape(-1,1), y7)

In [ ]:
leave_one_out_validation(x28.reshape(-1,1), y28)

In [ ]:
leave_one_out_validation(x90.reshape(-1,1), y90)

**First sight analysis:**

* There is few more we can improve at the 7th day using only the kaolinite content, data distribution is quite a straight line.
* For 1st and 3rd day the problem is more the sparsification of the points than the lack of expresivity of the model.
* For 28th and 90th day until 40% of kaolinite content the compression strength increases linearly and then estabilizes. Makes sense a non-linear model.
* It doesn't make sense in any model a degree 3 regression model, compression strength increases with kaolinite content.

#### Non-linear models based on the kaolinite content

In [ ]:
# x_2 = Kaolinite content, (Kaolinite content)^2
x1_2   = np.array([x1, x1**2]).T
x3_2   = np.array([x3, x3**2]).T
x7_2   = np.array([x7, x7**2]).T
x28_2   = np.array([x28, x28**2]).T
x90_2 = np.array([x90,x90*x90]).T 

In [ ]:
leave_one_out_validation(x1_2, y1)

In [ ]:
leave_one_out_validation(x3_2, y3)

In [ ]:
leave_one_out_validation(x7_2, y7)

In [ ]:
leave_one_out_validation(x28_2, y28)

In [ ]:
leave_one_out_validation(x90_2, y90)

**First sight analysis:**

* Expected results, better models obtained for 28th and 90th day compression strength obtained.
* We might be experiencing overfitting with this model for 1st and 3rd day measurements because we're not increasing the compressive strength with the increase of calonita for small contents.

### Feature selection

In [ ]:
#### TODO: Dummy issue, we have generalized feature selection function but in one dataset days ar "7D" and in the other days are "day_7"

In [ ]:
# Which features are realiable? Those with more than RELIABLE_THRESHOLD measurements
RELIABLE_THRESHOLD = 45

features = data_full_clay.columns[14:]
reliable_features = [f for f in features if data_full_clay[f].describe()[0] >= RELIABLE_THRESHOLD]

In [ ]:
# Show the reliable features, with enough points to rely
#reliable_features

In [ ]:
feature_selection(data_full_clay,reliable_features)
#feature_selection(features)

#### Linear regression models based on the kaolinite content and other features

Following the results obtained in the feature selection part, here we are creating and analyzing the models done with the kaolinite conent (in degree one and two) as well as other feature.

In [ ]:
### TODO: Same issue than in feature selection, unify days names in both datasets!

In [ ]:
# Day 1 
mod = smf.ols(formula='day_1 ~ Kaolinite_content + Kaolinite_content_square + Dv50', data=get_model_data(data_full_clay,'Dv50','1D'))
np.random.seed(2)
res = mod.fit()
res.summary()

In [ ]:
# Day 1 
mod = smf.ols(formula='day_1 ~ Kaolinite_content + Kaolinite_content_square + CaO', data=get_model_data('CaO','1D'))
np.random.seed(2)
res = mod.fit()
res.summary()

In [ ]:
# Day 3
mod = smf.ols(formula='day_3 ~ Kaolinite_content + Kaolinite_content_square + Dv50', data=get_model_data('Dv50','3D'))
np.random.seed(2)
res = mod.fit()
res.summary()

In [ ]:
# Day 3
mod = smf.ols(formula='day_3 ~ Kaolinite_content + Kaolinite_content_square + CaO', data=get_model_data('CaO','3D'))
np.random.seed(2)
res = mod.fit()
res.summary()

In [ ]:
# Day 7
mod = smf.ols(formula='day_7 ~ Kaolinite_content + Kaolinite_content_square + BET_specific_surface', data=get_model_data('BET_specific_surface','7D'))
np.random.seed(2)
res = mod.fit()
res.summary()

In [ ]:
# Day 28
mod = smf.ols(formula='day_2 ~ Kaolinite_content + Kaolinite_content_square + BET_specific_surface', data=get_model_data('BET_specific_surface','28D'))
np.random.seed(2)
res = mod.fit()
res.summary()

In [ ]:
# Day 28
mod = smf.ols(formula='day_2 ~ Kaolinite_content + Kaolinite_content_square + TiO2', data=get_model_data('TiO2','28D'))
np.random.seed(2)
res = mod.fit()
res.summary()

In [ ]:
# Day 90
mod = smf.ols(formula='day_9 ~ Kaolinite_content + Kaolinite_content_square + BET_specific_surface', data=get_model_data('BET_specific_surface','90D'))
np.random.seed(2)
res = mod.fit()
res.summary()

In [ ]:
# Day 90
mod = smf.ols(formula='day_9 ~ Kaolinite_content + Kaolinite_content_square + TiO2', data=get_model_data('TiO2','90D'))
np.random.seed(2)
res = mod.fit()
res.summary()

### Confidence analysis for the models

We know our model (function depending on the features provided) as well as some metrics to have an idea of how well our model fits our data (R2) and how is it behaving in practice with new data (MSE). The objetive of this section is to provide a more mathematical analysis of the confidence we can expect from our models.

#### Toy example for putting things into practice

We are going to start with the Kaolinite content based model for compression strength at day 90 to exemplify the tools and techniques we can use to estimate confidence intervals in our model.

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
data_90=data_full_clay[['day_90','Kaolinite_content']].dropna().copy()
data_90.insert(2, 'Kaolinite_content_square', data_90['Kaolinite_content']**2, True)

In [ ]:
# Day 90
mod = smf.ols(formula='day_90 ~ Kaolinite_content_square + Kaolinite_content', data=data_90)
np.random.seed(2)
res = mod.fit()
res.summary()

In [ ]:
plot_confidence_intervals(data_90, 90)

In [ ]:
prstd, iv_l, iv_u = wls_prediction_std(res)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

ax.plot(x90, y90, 'o', label="data")
#ax.plot(x90, model.predict(x90.reshape(-1,1)), 'b-', label="True")
ax.plot(x90, res.fittedvalues, 'r--.', label="OLS")
ax.plot(x90, iv_u, 'r--')
ax.plot(x90, iv_l, 'r--')
ax.legend(loc='best')

In [ ]:
data_28=data_full_clay[['day_28','Kaolinite_content']].dropna().copy()
data_28.insert(2, 'Kaolinite_content_square', data_28['Kaolinite_content']**2, True)

In [ ]:
plot_confidence_intervals(data_28, 28)

In [ ]:
prstd, iv_l, iv_u = wls_prediction_std(res)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

ax.plot(x28, y28, 'o', label="data")
#ax.plot(x90, model.predict(x90.reshape(-1,1)), 'b-', label="True")
ax.plot(x28, res.fittedvalues, 'r--.', label="OLS")
ax.plot(x28, iv_u, 'r--')
ax.plot(x28, iv_l, 'r--')
ax.legend(loc='best')

**First sight analysis:**

* We can't afford confidence intervals of such a high precission with such a lack of points, specially for high kaolinite contents.
* However, this worths a try after adding all the points from the second excel!

### References:

* How to deal with small datasets:\
https://medium.com/rants-on-machine-learning/what-to-do-with-small-data-d253254d1a89

* Feature engineering: \
https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/

* Feature selection: \
https://en.wikipedia.org/wiki/Feature_selection \
https://machinelearningmastery.com/an-introduction-to-feature-selection/ \
https://machinelearningmastery.com/feature-selection-machine-learning-python/ \
https://machinelearningmastery.com/feature-selection-in-python-with-scikit-learn/ \
https://machinelearningmastery.com/calculate-feature-importance-with-python/

* Linear regression with python: \
https://realpython.com/linear-regression-in-python/ 

* Confidence estimation: \
https://www.puneetarora2000.com/2020/01/data-interpretation-understanding-ols.html \
https://medium.com/@jyotiyadav99111/statistics-how-should-i-interpret-results-of-ols-3bde1ebeec01 \
https://www.statsmodels.org/stable/regression.html \

